In [4]:
#testing uploading pdf 
import PyPDF2 
def extract_pdf_content(content: bytes) -> str:
    try:
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(content))
        text = ""
        for i, page in enumerate(pdf_reader.pages):
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print('error with file')
        raise

In [3]:
pdf = 

0
